In [1]:
# need to install the package below
#pip install ipywidgets voila
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

In [2]:
%store -r best_model
%store -r submission_df
%store -r X_test

In [3]:
# Set Claim Indentifier as index of X_test
X_test = X_test.set_index(submission_df['Claim Identifier'])

# Define it as input 
claim_identifier_input = widgets.IntText(description='Claim Identifier:', layout=widgets.Layout(width='60%'))

# Output for displaying results
output = widgets.Output()

# CLAIM_TYPE_MAPPING dictionary (maps model output to readable claim type labels)
CLAIM_TYPE_MAPPING = {
    0: '1. CANCELLED',
    1: '2. NON-COMP',
    2: '3. MED ONLY',
    3: '4. TEMPORARY',
    4: '5. PPD SCH LOSS',
    5: '6. PPD NSL',
    6: '7. PTD',
    7: '8. DEATH'
}

In [4]:
# Function to validate the Claim Identifier and make the prediction
def validate_and_predict(change):
    claim_identifier = claim_identifier_input.value
    with output:
        output.clear_output()  # Clears previous outputs
        if claim_identifier not in X_test.index:
            print("Not a valid Claim ID")  # Handles invalid claims
        else:
            # Extract the features for the given claim identifier
            customer_features = X_test.loc[[claim_identifier]]
            
            # Make prediction using the model
            prediction = best_model.predict(customer_features)
            
            # Map the prediction to the corresponding claim type
            predicted_class = prediction[0]
            claim_type = CLAIM_TYPE_MAPPING.get(predicted_class, "Unknown Class")
            
            # Display the prediction result
            print(f"Prediction for Claim ID {claim_identifier}: {claim_type}")
            
            # Show the prediction probabilities as a bar chart
            display_prediction_probabilities(claim_identifier)

In [5]:
# Function to display prediction probabilities as a bar chart
def display_prediction_probabilities(claim_identifier):
    if claim_identifier in X_test.index:
        # Extract features for the claim
        customer_features = X_test.loc[[claim_identifier]]
        
        # Gets our model predicted probabilities
        probabilities = best_model.predict_proba(customer_features)[0]
        classes = best_model.classes_
        
        # Map classes to right labels
        class_labels = [CLAIM_TYPE_MAPPING.get(cls, f"Class {cls}") for cls in classes]
        
        # Plot probabilities
        plt.figure(figsize=(8, 5))
        plt.bar(class_labels, probabilities, color='skyblue')
        plt.title(f"Prediction Probabilities for Claim ID {claim_identifier}")
        plt.xlabel("Claim Type")
        plt.ylabel("Probability")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("Not a valid Claim ID")

In [9]:
# Try 6166047 for example

# Attach event listener to the input widget
claim_identifier_input.observe(validate_and_predict, names='value')

# Display the claim identifier input and output area
display(claim_identifier_input)
display(output)

IntText(value=6166047, description='Claim Identifier:', layout=Layout(width='60%'))

Output(outputs=({'name': 'stdout', 'text': 'Prediction for Claim ID 6166047: 2. NON-COMP\n', 'output_type': 's…

In [7]:
feature_sliders = {}
for column in X_test.columns:
    # As we scaled the data using standard scaler
    # we use range -3 and 3 as it contains most of the data
    slider = widgets.FloatSlider(
        value=0.0,  
        min=-3.0,   
        max=3.0,   
        step=0.01,  
        description=column
    )
    feature_sliders[column] = slider


In [8]:
def simulate_prediction(change):
    # Collect all slider values
    feature_values = np.array([slider.value for slider in feature_sliders.values()]).reshape(1, -1)
    
    # Get the prediction from the model
    prediction = best_model.predict(feature_values)
    claim_type = CLAIM_TYPE_MAPPING.get(prediction[0], "Unknown Class")
    
    # Display the prediction
    with output:
        output.clear_output()
        print(f"Prediction: {claim_type}")

# Output area for displaying results
output = widgets.Output()

# Link sliders to the simulate prediction function
for slider in feature_sliders.values():
    slider.observe(simulate_prediction, names="value")

# Display sliders and output area
display(widgets.VBox(list(feature_sliders.values())))
display(output)

Output()